## Setup

In [15]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import pandas as pd
import harp

from harp_resources import process, utils
from sleap import load_and_process as lp

# Extracting Noras data

In [16]:
photometry_path = '/Volumes/RanczLab/Nora_Cohort1_training/Training_day4/B6J2717-2024-11-28T09-37-55/photometry'
data_path = '/Volumes/RanczLab/Nora_Cohort1_training/Training_day4/B6J2717-2024-11-28T09-37-55/'

photometry_path = '/Volumes/RanczLab/Nora_Cohort1_training/Training_day4/B6J2717-2024-11-28T09-37-55/photometry'
data_path = '/Volumes/RanczLab/Nora_Cohort1_training/Training_day4/B6J2717-2024-11-28T09-37-55/'

h1_datafolder = data_path+'HarpDataH1'
h2_datafolder = data_path+'HarpDataH2'

In [17]:
pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv') #Processed_fluorescence.csv #Fluorescence.csv

,TimeStamp,470_dfF,560_dfF,410_dfF,z_470,z_560,z_410,Time,mouseID,Area,Sex
0,15.011111,-0.140807,1.509909,-0.444041,-0.301599,0.916605,-0.830532,2024-11-28 10:43:05.011111,B6J2717,V2M,M
1,15.044443,-0.160119,1.496179,-0.471740,-0.336486,0.909917,-0.879771,2024-11-28 10:43:05.044443,B6J2717,V2M,M
2,15.077780,-0.169650,1.706411,-0.505553,-0.353703,1.012305,-0.939881,2024-11-28 10:43:05.077780,B6J2717,V2M,M
3,15.111109,-0.167552,2.075621,-0.543008,-0.349913,1.192120,-1.006466,2024-11-28 10:43:05.111109,B6J2717,V2M,M
4,15.144446,-0.156654,2.027079,-0.579149,-0.330225,1.168477,-1.070713,2024-11-28 10:43:05.144446,B6J2717,V2M,M
...,...,...,...,...,...,...,...,...,...,...,...
27220,922.219880,-0.776283,0.695412,-0.670457,-1.399628,0.502656,-1.117508,2024-11-28 10:58:12.219880,B6J2717,V2M,M
27221,922.253214,-0.775014,0.777530,-0.694914,-1.397416,0.540610,-1.156774,2024-11-28 10:58:12.253214,B6J2717,V2M,M
27222,922.286536,-0.771584,0.478150,-0.733053,-1.391440,0.402235,-1.218007,2024-11-28 10:58:12.286536,B6J2717,V2M,M
27223,922.319870,-0.771993,0.491793,-0.787642,-1.392150,0.408540,-1.305652,2024-11-28 10:58:12.319870,B6J2717,V2M,M


In [19]:
# from andrew:
#Changed didgital output file
h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)
session_data_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
photometry_reader = utils.PhotometryReader("Fluorescence")
video_reader = utils.Video("VideoData1")
onix_digital_reader = utils.TimestampedCsvReader("OnixDigital", columns=["Clock", "HubClock", 
                                                                         "DigitalInputs0",
                                                                         "DigitalInputs1",
                                                                         "DigitalInputs2",
                                                                         "DigitalInputs3",
                                                                         "DigitalInputs4",
                                                                         "DigitalInputs5"
                                                                         "DigitalInputs6",
                                                                         "DigitalInputs7",
                                                                         "DigitalInputs8",
                                                                         "Buttons"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

In [20]:
# read metadata
session_settings = utils.load(session_data_reader, data_path)

print(session_settings.iloc[0]['metadata'].blocks[0].haltProtocol)

DotMap(randomDelay=1.5, minumumDelay=0.1, haltGain=0.0, haltMotorGain=1.0, haltTime=2.0)


In [21]:
# read harp streams, experiment events, video
flow_sensor = utils.load_harp(h1_reader.OpticalTrackingRead, h1_datafolder)
camera_triggers = utils.load_harp(h1_reader.Cam0Event, h1_datafolder)
experiment_events = utils.load(experiment_events_reader, root_folder)
video_data = utils.load(video_reader, root_folder)

OverflowError: Python integer 2182416 out of bounds for uint8

# Bulk extraction
## old data

In [2]:
#data_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/MMclosed&Regular_120824/2024-08-12T16-51-16_B3M3')
#photometry_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/MMclosed&Regular_120824/photometry/B3M3_MMclosed&Regular_day1/2024_08_12-18_57_17')


In [3]:
rootdir = '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824' #Enter root path
data_paths = []
photometry_paths= []
for dirpath, subdirs, files in os.walk(rootdir):
    if 'ExperimentEvents' in dirpath:
        data_paths.append(dirpath[:-17])
    for x in files:
        if 'Processed_fluorescence.csv' in x:
            photometry_paths.append(dirpath)





In [4]:
photometry_paths

['/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T09-46-19_B2M5/photometry',
 '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T09-46-19_B2M5/photometry',
 '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T10-40-45_B2M4/photometry',
 '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T11-45-03_B3M3/photometry',
 '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T13-40-07_B3M1/photometry',
 '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T14-22-22_B3M2/photometry']

In [5]:
data_paths

['/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T09-46-19_B2M5',
 '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T10-40-45_B2M4',
 '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T11-45-03_B3M3',
 '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T13-40-07_B3M1',
 '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T14-22-22_B3M2']

In [12]:
def run_extraction_makeh5(data_path, photometry_path, make_h5 = False):
    print(f'\n Running extraction for {data_path.split('/')[-1][20:]} for session {data_path.split('/')[-2]}')
    #data_path = Path(data_path)
    #photometry_path = Path(photometry_path)
    SessionSettings = utils.read_SessionSettings(Path(data_path), print_contents=True)
    conversions = process.calculate_conversions_second_approach(Path(data_path), Path(photometry_path), verbose=False)
    streams = utils.load_registers(Path(data_path))

    Photometry = utils.read_fluorescence(Path(photometry_path))
    Photometry['HARP Timestamps'] = conversions['photometry_to_harp_time'](Photometry['TimeStamp'])
    
    OnixAnalogClock = utils.read_OnixAnalogClock(Path(data_path))
    OnixAnalogData = utils.read_OnixAnalogData(Path(data_path), binarise=True)
    ExperimentEvents = utils.read_ExperimentEvents(Path(data_path)) 
    
    photodiode_series = pd.Series(OnixAnalogData[:,0], index=conversions['onix_to_harp_timestamp'](OnixAnalogClock))

    print('Adding Photometry, Eye Movements and Photodiode to the streams')
    streams = process.reformat_and_add_many_streams(streams, Photometry, 'Photometry', ['CH1-410', 'CH1-470', 'CH1-560'], index_column_name='HARP Timestamps')
    streams = process.add_stream(streams, 'ONIX', photodiode_series, 'Photodiode')
    
    
    _ = process.get_timepoint_info(streams, print_all=True)
    resampled_streams = process.pad_and_resample(streams, resampling_period='1 ms', method='linear')
    _ = process.get_timepoint_info(resampled_streams, print_all=True)

    print('Applying linear and angular conversion to Optical tracking sensor streams (cm / sec and degrees / sec)')
    resampled_streams['H1']['OpticalTrackingRead0X(46)'] = process.running_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0X(46)']*100)
    resampled_streams['H1']['OpticalTrackingRead0Y(46)'] = process.rotation_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0Y(46)'])

    print('Streams are extracted and can be used or made to h5')
    if make_h5:
        process.save_streams_as_h5(Path(data_path), resampled_streams, streams_to_save_pattern, SessionSetting)

In [13]:
data_path, resampled_streams, streams_to_save_pattern =run_extraction_makeh5(data_paths[1], photometry_paths[1]) 



 Running extraction for B2M4 for session G8_MMclosed-and-open_070824
{
    "seconds": 685478.300256,
    "value": {
        "motorSettings": {
            "nominalPulseInterval": 200,
            "initialPulseInterval": 200,
            "pulseStepInterval": 2,
            "pulsePeriod": 10
        },
        "blocks": [
            {
                "alias": "LinearMismatch",
                "playbackFilePath": "",
                "blockGainModifier": 1.0,
                "flowXToVisualGain": 0.0008,
                "flowYToVisualGain": 0.0,
                "rotaryToVisualGain": 0.0,
                "playbackToVisualGain": 0.0,
                "stimulus": {
                    "type": 0,
                    "spatialFrequency": 28.8,
                    "offset1": -90.0,
                    "extent1": 180.0,
                    "offset2": 90.0,
                    "extent2": 180.0,
                    "path": ""
                },
                "flowXToMotorGain": 0.0,
              

/Users/hildeteigen/Documents/GitHub/vestibular_vr_pipeline/harp_resources/utils.py:143: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sorted_filenames = pd.to_datetime(pd.Series([x.split('_')[1].split('.')[0] for x in filenames])).sort_values()



Shape of OnixAnalogClock == [250166000] shape of OnixAnalogFrameCount == [250168300].
Cutting 2300 values from the beginning of OnixAnalogFrameCount. Data considered to be MISSING.



/Users/hildeteigen/Documents/GitHub/vestibular_vr_pipeline/harp_resources/utils.py:134: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sorted_filenames = pd.to_datetime(pd.Series([x.split('_')[1].split('.')[0] for x in filenames])).sort_values()


OnixDigital has 818 events and Events.csv has 650 events. Cutting to the minimum 650 events from the beggining, aligning by the end.

Calculation of conversions finished in 1108.85 seconds.
Dataset 2024-08-07T10-40-45_B2M4 contains following registers:
H1: [32, 33, 46]
H2: [38, 39, 42]


AttributeError: 'RegisterReader' object has no attribute 'pattern'

In [ ]:
data_path

## Loading and Synchronisation 

In [17]:
photometry_path = Path(photometry_paths[3])
data_path = Path(data_paths[3])
photometry_paths[3]

'/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_070824/2024-08-07T11-45-03_B3M3/photometry'

In [10]:
photometry_path = Path('/Volumes/RanczLab/Photometry_recordings/August_Mismatch_Experiment_G8m/B3M3_MMclosed_and_Regular_day2')
data_path = Path('/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-regular_130824/2024-08-13T12-53-01_B3M3')

In [11]:
photometry_path = Path('/Volumes/RanczLab/Photometry_recordings/August_Mismatch_Experiment_G8m/B2M4_MMclosed_and_open_day2/2024_08_08-11_24_00')
data_path = Path('/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-open_080824/2024-08-08T09-20-54_B2M4')
#works

In [19]:
conversions = process.calculate_conversions_second_approach(data_path, photometry_path, verbose=False)
# After hardware ONIX clock implementation - this will have to be adapted
# Only photometry will need to be converted

OnixAnalogClock loaded in 5.77 seconds.

Shape of OnixAnalogClock == [205033700] shape of OnixAnalogFrameCount == [205036100].
Cutting 2400 values from the beginning of OnixAnalogFrameCount. Data considered to be MISSING.

OnixDigital has 670 events and Events.csv has 673 events. Cutting to the minimum 670 events from the beggining, aligning by the end.

Calculation of conversions finished in 77.52 seconds.


In [20]:
streams = utils.load_registers(data_path)

Photometry = utils.read_fluorescence(photometry_path)
Photometry['HARP Timestamps'] = conversions['photometry_to_harp_time'](Photometry['TimeStamp'])

OnixAnalogClock = utils.read_OnixAnalogClock(data_path)
OnixAnalogData = utils.read_OnixAnalogData(data_path, binarise=True)
ExperimentEvents = utils.read_ExperimentEvents(data_path) 



photodiode_series = pd.Series(OnixAnalogData[:,0], index=conversions['onix_to_harp_timestamp'](OnixAnalogClock))

Dataset 2024-08-07T13-40-07_B3M1 contains following registers:
H1: [32, 33, 46]
H2: [39, 42]


OverflowError: Python integer 1601990 out of bounds for uint8

In [22]:
# Adding Photometry, Eye Movements and Photodiode to the streams
streams = process.reformat_and_add_many_streams(streams, Photometry, 'Photometry', ['CH1-410', 'CH1-470', 'CH1-560'], index_column_name='HARP Timestamps')
streams = process.add_stream(streams, 'ONIX', photodiode_series, 'Photodiode')

In [23]:
_ = process.get_timepoint_info(streams, print_all=True)
resampled_streams = process.pad_and_resample(streams, resampling_period='1 ms', method='linear')
_ = process.get_timepoint_info(resampled_streams, print_all=True)

Global first timestamp: 1904-01-09 20:51:29.164480
Global last timestamp: 1904-01-09 21:17:44.905083676
Global length: 0 days 00:26:15.740603676

H1
Cam0Event(32): 
	first  1904-01-09 20:51:29.166560 
	last   1904-01-09 21:17:40.952480 
	length 0 days 00:26:11.785920 
	mean difference between timestamps 0 days 00:00:00.017096522
Cam1Event(33): 
	first  1904-01-09 20:51:29.166560 
	last   1904-01-09 21:17:40.952512 
	length 0 days 00:26:11.785952 
	mean difference between timestamps 0 days 00:00:00.017096523
StartCam0(38): 
	first  1904-01-09 20:53:20.413504 
	last   1904-01-09 20:53:59.238496 
	length 0 days 00:00:38.824992 
	mean difference between timestamps 0 days 00:00:12.941664
StartCam1(38): 
	first  1904-01-09 20:53:20.413504 
	last   1904-01-09 20:53:59.238496 
	length 0 days 00:00:38.824992 
	mean difference between timestamps 0 days 00:00:12.941664
StopCam0(38): 
	first  1904-01-09 20:53:20.413504 
	last   1904-01-09 20:53:59.238496 
	length 0 days 00:00:38.824992 
	mean diff

In [27]:
# Applying linear and angular conversion to Optical tracking sensor streams
# OpticalTrackingRead0X(46) converted to centimeters per second
# OpticalTrackingRead0Y(46) covnerted to degrees per second
resampled_streams['H1']['OpticalTrackingRead0X(46)'] = process.running_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0X(46)']*100)
resampled_streams['H1']['OpticalTrackingRead0Y(46)'] = process.rotation_unit_conversion(resampled_streams['H1']['OpticalTrackingRead0Y(46)'])

In [28]:
print(type(resampled_streams['Photometry']['CH1-470']))
resampled_streams['Photometry']['CH1-470']

<class 'pandas.core.series.Series'>


1904-01-09 20:51:29.164     0.000000
1904-01-09 20:51:29.165     0.019158
1904-01-09 20:51:29.166     0.038317
1904-01-09 20:51:29.167     0.057475
1904-01-09 20:51:29.168     0.076633
                             ...    
1904-01-09 21:17:44.901    69.284647
1904-01-09 21:17:44.902    69.277735
1904-01-09 21:17:44.903    69.270824
1904-01-09 21:17:44.904    69.263912
1904-01-09 21:17:44.905    69.257000
Freq: ms, Length: 1575742, dtype: float64

In [29]:
streams_to_save_pattern = {'H1': ['OpticalTrackingRead0X(46)', 'OpticalTrackingRead0Y(46)'], 'H2': ['Encoder(38)'], 'Photometry': ['CH1-410', 'CH1-470', 'CH1-560'], 'ONIX': ['Photodiode']}
streams_to_save_pattern


{'H1': ['OpticalTrackingRead0X(46)', 'OpticalTrackingRead0Y(46)'],
 'H2': ['Encoder(38)'],
 'Photometry': ['CH1-410', 'CH1-470', 'CH1-560'],
 'ONIX': ['Photodiode']}